# Time-series ICESat1 & 2

1. Plotting time-series ICESat 1
2. Plotting time-series ICESat 1 & 2
3. Plotting them using box and whisker plot

## Plotting time-series ICESat 1

### Load the ICESat 1

In [1]:
import os
from pathlib import Path
import h5py
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import time
from astropy.time import Time
import datetime
%matplotlib widget
%load_ext autoreload
%autoreload 2

In [2]:
def glah14_to_df(filename,bounds):
    ## sp_ex = [103.643, 104.667, 12.375, 13.287]
    ## Bounds are [Longitude_West, Longitude_East, Latitude_South, Latitude_North]
    f = h5py.File(filename, 'r')
    lat = f['Data_40HZ']['Geolocation']['d_lat'][:]
    lon = f['Data_40HZ']['Geolocation']['d_lon'][:]
    elev = f['Data_40HZ']['Elevation_Surfaces']['d_elev'][:]
    sec = f['Data_40HZ']['Elevation_Corrections']['d_satElevCorr'][:]
    scf = f['Data_40HZ']['Quality']['sat_corr_flg'][:]
    satndx = f['Data_40HZ']['Quality']['i_satNdx'][:]
    dem = f['Data_40HZ']['Geophysical']['d_DEM_elv'][:]
    date_gps = f['Data_40HZ']['DS_UTCTime_40'][1] + 630763213 
    date_ymd = Time(date_gps,format='gps').datetime.strftime("%Y-%m-%d")
    
    glah14_df = pd.DataFrame({'Latitude':lat,'Longitude':lon,'Elevation':elev,
                            's_El_Corr':sec, 's_Corr_f':scf,'in_sat':satndx,
                            'DEM':dem,'Date':date_ymd})
    #### Subsetting
    glah14_df_subset = glah14_df.loc[(glah14_df['Longitude']>=bounds[0]) 
                          & (glah14_df['Longitude']<=bounds[1])
                          & (glah14_df['Latitude']>=bounds[2])
                          & (glah14_df['Latitude']<=bounds[3])]
    return glah14_df_subset

In [3]:
##### load files
## set the directory
data_home = Path('/home/jovyan/ICESat_water_level/extraction/icesat/')
## list them up and check them
files= list(data_home.glob('*.H5'))
### Spatial Bounds: 
tsl_sp_ex = [103.643, 104.667, 12.375, 13.287]
### load the water mask
tsl_wm = gpd.read_file('/home/jovyan/ICESat_water_level/extraction/shp/tsl_sample_dis.shp') ## water mask: Tonle Sap Lake

In [4]:
#### convert to geoDF and clip it. 
is1_lst = []
for num in range(1,len(files)):
    gdf = gpd.GeoDataFrame(glah14_to_df(files[num],tsl_sp_ex))
    gdf.set_geometry(
        gpd.points_from_xy(gdf['Longitude'], gdf['Latitude']),
        inplace=True, crs='EPSG:4326')
    gdf_wm = gpd.clip(gdf, tsl_wm)
    is1_lst.append(gdf_wm)

### plotting one track

In [5]:
test2 = is1_lst[0]
fig=plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.plot(test2['Date'],test2['Elevation'],'.',markersize=0.25, label='all segements')
h_leg=ax.legend()
plt.title('Water Surface_ICESat1')
ax.set_xlabel('Date')
ax.set_ylabel('Water Surface, m')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Removing outliers and plot it.

In [7]:
### outliers
def out_rmv(df,var):
    ### lower (25%)
    lwq = df[var].quantile(0.25)
    ### upper (75%)
    hwq = df[var].quantile(0.75)

    ### IQR
    iqr = hwq - lwq

    lw_out = lwq-1.5*iqr
    hw_out = hwq+1.5*iqr

    ###LSHoutlier > q0.75 + 1.5 ∗ IQR or LSHoutlier < q0.25 − 1.5 ∗ IQR
    return df.loc[(df[var] >= lw_out) & (df[var] <= hw_out)]

In [16]:
#### convert to geoDF and clip it. 
gl14_df = pd.DataFrame()
date_list = db['Date_num'].unique()
for num in range(0,3):
    df2pd = pd.DataFrame(out_rmv(is1_lst[num],'Elevation'))
    gl14_df.append(df2pd, ignore_index = True)
    print(num)
print(gl14_df)

0
1
2
Empty DataFrame
Columns: []
Index: []


In [15]:
pd.DataFrame(is1_lst[0])

,Latitude,Longitude,Elevation,s_El_Corr,s_Corr_f,in_sat,DEM,Date,geometry
159593,13.167806,104.526181,16.961,1.797693e+308,4,16,25.06,2003-11-04,POINT (104.52618 13.16781)
159875,12.728531,104.464596,-7.943,0.000000e+00,0,0,-10.73,2003-11-04,POINT (104.46460 12.72853)
159876,12.726976,104.464378,-7.803,0.000000e+00,0,0,-10.70,2003-11-04,POINT (104.46438 12.72698)
159877,12.725422,104.464161,-7.890,0.000000e+00,0,0,-10.65,2003-11-04,POINT (104.46416 12.72542)
159878,12.723868,104.463944,-7.868,0.000000e+00,0,0,-10.58,2003-11-04,POINT (104.46394 12.72387)
...,...,...,...,...,...,...,...,...,...
159980,12.564955,104.441702,-7.743,0.000000e+00,0,0,-11.11,2003-11-04,POINT (104.44170 12.56495)
159982,12.561846,104.441274,-7.729,0.000000e+00,0,0,-9.36,2003-11-04,POINT (104.44127 12.56185)
159983,12.560292,104.441058,-7.768,0.000000e+00,0,0,-9.19,2003-11-04,POINT (104.44106 12.56029)
159984,12.558739,104.440841,-7.789,0.000000e+00,0,0,-9.00,2003-11-04,POINT (104.44084 12.55874)


In [ ]:
### I made a function for stacking the information
def alt13_a_out(db,var2):
    alt13_df = pd.DataFrame()
    date_list = db['Date_num'].unique()
    for prd in date_list:
        s_db = db.loc[db['Date_num']==prd]
        beam_lst = db['Beam'].unique()
        for beam_type in beam_lst:
            df_select = s_db.loc[s_db['Beam']==beam_type]
            alt13_df = alt13_df.append(out_rmv(df_select,var2), ignore_index = True)
    return alt13_df

In [ ]:
is1_lst_out['Date'][:]

### Plottting time-series water level changes

In [ ]:
fig=plt.figure(figsize=(10,4))
ax = fig.add_subplot(111)
ax.grid()
ax.xaxis.set_major_locator(plt.MaxNLocator(5))
ax.set_ylim([-20,0])
ax.plot(a_tracks['Date'],a_tracks['SurfaceH'],'.', color = 'orange', markersize=0.25, label='all segements')
ax.plot(a_tracks_out['Date'],a_tracks_out['SurfaceH'],'.', color = 'green',markersize=0.4, label='Without outliers')
##ax.plot(av_h['Date'],av_h['Av_level'],'+', color = 'red',markersize=3, label='Averaged')
##ax.plot(av_h['Date'],av_h['Av_level'],color = 'red',linewidth=0.25)
h_leg=ax.legend()
plt.title('Water Surface')
ax.set_xlabel('Date')
ax.set_ylabel('Water Surface, m')
plt.show()